In [83]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from matplotlib import cm
import random
import matplotlib.gridspec as gridspec
import sys
import copy


dim = 5

flatTerrainRate = 0.1
hillyTerrainRate = 0.3
forestedTerrainRate = 0.7
caveTerrainRate = 0.9

flat = 0 
hilly = 1
forested = 2
cave = 3

# Generates the terrain and values for the map
def generateMap(dim):
        map = np.zeros([dim, dim])
        for i in range(dim):
            for j in range(dim):
                p = np.random.rand()
                if p <= 0.2:
                    # Initialize Flat Terrain
                    map[i][j] = flat
                elif p > 0.2 and p <= 0.5:
                    # Initialize Hilly Terrain
                    map[i][j] = hilly
                elif p > 0.5 and p <= 0.8:
                    # Initialize Forested Terrain
                    map[i][j] = forested
                else:
                    # Initialize Cave Terrain
                    map[i][j] = cave
        return map

# Generates the coordinates of the target marker and agent marker
def generateMarker(dim):
    x = np.random.randint(dim)
    y = np.random.randint(dim)
    return [x, y]

# Generates the initial probability state of the grid 
def generateInitialState(dim):
    initialProb = 1/(dim*dim)
    probState = np.zeros([dim, dim])
    for i in range(dim):
        for j in range(dim):
            probState[i][j] = initialProb
    return probState

# State where agent is at target coordinate and tries to find the target
def targetState():
    foundVal = np.random.rand()

    # If the randomly generated number is greater than the rate of the target then the target is found
    if map[agent[0]][agent[1]] == flat :
        if foundVal > flatTerrainRate :
            print("finish flat")
            print(t)
            sys.exit(0)
    elif map[agent[0]][agent[1]] == hilly :
       if foundVal > hillyTerrainRate :
            print("finish hilly")
            print(t)
            sys.exit(0)
    elif map[agent[0]][agent[1]] == forested :
       if foundVal > forestedTerrainRate :
            print("finish forested")
            print(t)
            sys.exit(0)
    else :
        if foundVal > caveTerrainRate :
            print("finish cave")
            print(t)
            sys.exit(0)
    
    # if not found we continue with update
    return
        

# Updates our current belief state at time t
def updateBeliefState(dim, beliefState):
    x = agent[0]
    y = agent[1]

    print("x:",agent[0])
    print("y:",agent[1])
    if (agent[0] == target[0] and agent[1] == target[1]):
        targetState()

    beliefState[x][y]

    if map[x][y] == flat :
        # currentProb = flatTerrainRate * initialProb
        beliefState[x][y] = beliefState[x][y] * flatTerrainRate
    elif map[x][y] == hilly :
        # currentProb = hillyTerrainRate * initialProb
        beliefState[x][y] = beliefState[x][y] * hillyTerrainRate
    elif map[x][y] == forested :
        # currentProb = forestedTerrainRate * initialProb
        beliefState[x][y] = beliefState[x][y] * forestedTerrainRate
    else :
        # currentProb = caveTerrainRate * initialProb
        beliefState[x][y] = beliefState[x][y] * caveTerrainRate

    
    newTotal = np.sum(beliefState)

    #newProbOfCurr = currentProb / newTotal
    #newProbOfRest = initialProb / newTotal

    #beliefState[0:dim][0:dim] = newProbOfRest
    #beliefState[x][y] = newProbOfCurr
    beliefState = beliefState/newTotal
    

# Searches grid for next position that agent shall search
def searchNext(t,dim, beliefState):
    print("this is t",t)
    maxVal = 0
    mXCoor = 0
    mYCoor = 0
    print("beliefState change in search:", beliefState)
    for i in range(dim):
        for j in range(dim):
            print("this is prob:", beliefState[i][j], "this is max value:", maxVal)
            if beliefState[i][j] > maxVal:
                print("here")
                maxVal = beliefState[i][j]
                mXCoor = i
                mYCoor = j
    agent[0] = mXCoor
    agent[1] = mYCoor

# Runs the search for the agent
def executeRule1(beliefState):
    t = 0
    while True:
        t = t + 1
        updateBeliefState(dim,beliefState)
        searchNext(t,dim,beliefState)






    


# Initialize map, target, agent, and initial probability state of each cell
map = generateMap(dim)
target = generateMarker(dim)
agent = generateMarker(dim)
beliefState = generateInitialState(dim)
print(map)
print(target[0])
print(target[1])
executeRule1(beliefState)
print(beliefState)
grid = gridspec.GridSpec(ncols=2, nrows=2)

# Make a grid of 4 equal parts
fig = plt.figure(figsize=(15,15))
ogMap = fig.add_subplot(grid[0, 0])
Image = ogMap.imshow(map, cmap = cm.get_cmap('Greens', 4))
ogMap.set_title("Original Map")
ogMap.scatter(target[0], target[1], s=100, c='red', marker='x')


99997 this is max value: 0.032400000000000005
this is prob: 0.004 this is max value: 0.032400000000000005
this is prob: 0.032400000000000005 this is max value: 0.032400000000000005
this is prob: 0.012 this is max value: 0.032400000000000005
this is prob: 0.027999999999999997 this is max value: 0.032400000000000005
this is prob: 0.032400000000000005 this is max value: 0.032400000000000005
x: 0
y: 1
this is t 31
beliefState change in search: [[0.012   0.02916 0.028   0.004   0.004  ]
 [0.004   0.028   0.028   0.012   0.012  ]
 [0.028   0.012   0.012   0.012   0.012  ]
 [0.028   0.028   0.0324  0.0324  0.028  ]
 [0.004   0.0324  0.012   0.028   0.0324 ]]
this is prob: 0.012 this is max value: 0
here
this is prob: 0.029160000000000005 this is max value: 0.012
here
this is prob: 0.027999999999999997 this is max value: 0.029160000000000005
this is prob: 0.004 this is max value: 0.029160000000000005
this is prob: 0.004 this is max value: 0.029160000000000005
this is prob: 0.004 this is max va

SystemExit: 0